In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu
import statsmodels.api as sm

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [ ]:
file='GA_soy_7'; 
id=142

r = uu.deserialize(file,comment=False)
m = r['model'][id]

# Get the data
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope)

# Elaborate the data
wws = um.var_windows_from_cols(m.params.index)
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= wws)
model_df = pd.concat([raw_data['yield'], model_df], sort=True, axis=1, join='inner')

y_col  ='Yield'
y_df = model_df[[y_col]]


cols= [c for c in m.params.index if c != 'const']
X_df=model_df[cols]

folds = um.folds_expanding(model_df=model_df, min_train_size=10)

cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

print(cv_score.keys())

comp_list =['cv_corr', 'cv_p', 'cv_r_sq', 'cv_MAE', 'cv_MAPE']

for k in comp_list:
    print()
    print('Saved:', k, np.mean(r[k][id]))
    print('Calc :', k, np.mean(cv_score[k]))